<a href="https://colab.research.google.com/github/dakilaledesma/Atomized_NN_Experiments/blob/main/architecture_differences/Sequencer2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 121 ms, sys: 28.3 ms, total: 149 ms
Wall time: 19.6 s


In [2]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10686, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 10686 (delta 105), reused 158 (delta 83), pack-reused 10469
Receiving objects: 100% (10686/10686), 20.36 MiB | 24.04 MiB/s, done.
Resolving deltas: 100% (7818/7818), done.


In [3]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model sequencer2d_l --opt adabelief --lr 0.0001 --epochs 80 --aa v0r-mstd0.5 --aug-splits 2 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 600 600 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/ 

Streaming output truncated to the last 5000 lines.
Train: 12 [2350/2354 (100%)]  Loss: 3.218 (2.66)  Time: 0.457s,   17.51/s  (0.467s,   17.12/s)  LR: 9.460e-05  Data: 0.005 (0.008)
Train: 12 [2353/2354 (100%)]  Loss: 1.767 (2.66)  Time: 0.451s,   17.73/s  (0.467s,   17.12/s)  LR: 9.460e-05  Data: 0.000 (0.008)
Test: [   0/2354]  Time: 0.618 (0.618)  Loss:  1.5049 (1.5049)  Acc@1: 100.0000 (100.0000)  Acc@5: 100.0000 (100.0000)
Test: [  50/2354]  Time: 0.162 (0.147)  Loss:  3.4004 (1.1788)  Acc@1: 25.0000 (77.9412)  Acc@5: 25.0000 (93.1373)
Test: [ 100/2354]  Time: 0.132 (0.140)  Loss:  2.6074 (1.3764)  Acc@1: 25.0000 (71.2871)  Acc@5: 75.0000 (92.0792)
Test: [ 150/2354]  Time: 0.113 (0.138)  Loss:  0.2126 (1.2058)  Acc@1: 100.0000 (73.5099)  Acc@5: 100.0000 (94.3709)
Test: [ 200/2354]  Time: 0.104 (0.137)  Loss:  2.7656 (1.1242)  Acc@1:  0.0000 (75.6219)  Acc@5: 75.0000 (94.9005)
Test: [ 250/2354]  Time: 0.195 (0.137)  Loss:  0.2137 (1.1865)  Acc@1: 100.0000 (74.1036)  Acc@5: 100.0000

In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model sequencer2d_l --opt adabelief --lr 0.0001 --epochs 80 --aa v0r-mstd0.5 --aug-splits 2 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 600 600 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/ --resume drive/MyDrive/UNC/H2022/output/20220721-160046-sequencer2d_l-600/last.pth.tar

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth)
Downloading: "https://github.com/okojoalg/sequencer/releases/download/weights/sequencer2d_l.pth" to /root/.cache/torch/hub/checkpoints/sequencer2d_l.pth
Model sequencer2d_l created, param count:54028716
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225